In [10]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, CuDNNLSTM
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback


In [11]:
# new_model = keras.models.load_model('ltsm_test.h5')
auc_scores = []

In [12]:
path = '../data/'
comp = 'kaggle ltsm/'
EMBEDDING_FILE=f'glove.6B.50d.txt'
TRAIN_DATA_FILE=f'train.csv'
TEST_DATA_FILE=f'test.csv'

#  provide your path
BASELINE_DATA = 'bias_madlibs_89k.csv'

# try different thresholds
THRESHOLD = 0.5

In [13]:
def get_simple_label(results, threshold):
    results = np.array(results)
    if results[np.where( results > threshold )].size > 0:
        return "BAD"
    return "NOT_BAD"

In [14]:
def eval_artifitial(data_set, labels, model, tokenizer):
    list_tokenized = tokenizer.texts_to_sequences(data_set)
    X_t = pad_sequences(list_tokenized, maxlen=maxlen)
    
    y =  model.predict(X_t, batch_size=1024, verbose=1)
    scores = {"correct":0, "wrong":0}
    
    for i in range(len(y)):
        simple_label = get_simple_label(y[i], THRESHOLD)
        actual_label = labels[i]
        
        if simple_label == actual_label:
            scores["correct"] = scores["correct"] + 1
        else:
            scores["wrong"] = scores["wrong"] + 1 
            
    return scores["correct"] / len(y)

In [15]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [16]:
test = pd.read_csv(TEST_DATA_FILE)
train = pd.read_csv(TRAIN_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)



In [17]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf-8"))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

C:\Users\10218\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


In [18]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            auc_scores.append(score)


In [20]:
[X_tra, X_val, y_tra, y_val] = train_test_split(X_t, y, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

C:\Users\10218\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [21]:
hist = model.fit(X_tra, y_tra, batch_size=32, epochs=2, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 151592 samples, validate on 7979 samples
Epoch 1/2


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node bidirectional_1/CudnnRNN (defined at C:\Users\10218\Anaconda3\lib\site-packages\keras\layers\cudnn_recurrent.py:517) with these attrs: [is_training=true, seed2=0, input_mode="linear_input", T=DT_FLOAT, dropout=0, rnn_mode="lstm", direction="unidirectional", seed=87654321]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[node bidirectional_1/CudnnRNN (defined at C:\Users\10218\Anaconda3\lib\site-packages\keras\layers\cudnn_recurrent.py:517) ]]

Caused by op 'bidirectional_1/CudnnRNN', defined at:
  File "C:\Users\10218\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\10218\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\10218\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\10218\Anaconda3\lib\site-packages\tr

In [19]:
acc_hist = hist.history['acc']
acc_hist = np.array(acc_hist)
loss_hist = np.array(hist.history['loss'])

np.savetxt("acc_history.txt", acc_hist, delimiter=",")
np.savetxt("loss_history.txt", loss_hist, delimiter=",")

In [18]:
model.save("LSTM_cuDnn.hm5")

In [23]:
# Read the data
baseline_data = pd.read_csv(BASELINE_DATA, ",")
# get texts and labels
baseline_data_text = baseline_data["Text"].values[:5000]
baseline_labels = baseline_data["Label"].values[:5000]

In [24]:
eval_artifitial(baseline_data_text, baseline_labels, model, tokenizer)

5000/5000 [==============================] - 0s 26us/step


0.9086

In [38]:
y_test = new_model.predict([X_te], batch_size=1024, verbose=1)


153164/153164 [==============================] - 57s 371us/step


In [44]:
np.set_printoptions(precision=10, suppress=True)
print(len(np.where( y_test > 0.5 )))
# "{:.3f}".format(y_test[1])

2


In [47]:
result_classes = []

for item in y_test:
    result_classes.append(list_classes[np.argmax(item)])
    


["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
 '== From RfC == \n\n The title is fine as it is, IMO.'
 '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'
 ":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
 "I don't anonymously edit articles at all."
 'Thank you for understanding. I think very highly of you and would not revert without discussion.'
 'Please do not add nonsense to Wikipedia. Such edits are considered vandalism and quickly undone. If you would like to experiment, please use the sandbox in

In [51]:
# print(list_sentences_test[:10])
# print("msg: {0} class: {1}".format(list_sentences_test[:10], result_classes[:10]))

msg: ["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
 '== From RfC == \n\n The title is fine as it is, IMO.'
 '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "'
 ":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
 "I don't anonymously edit articles at all."
 'Thank you for understanding. I think very highly of you and would not revert without discussion.'
 'Please do not add nonsense to Wikipedia. Such edits are considered vandalism and quickly undone. If you would like to experiment, please use the sandb

In [57]:
df_tr = pd.read_csv(TRAIN_DATA_FILE)
df_t = pd.read_csv(TEST_DATA_FILE)



,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [ ]:
df_t.head(20)